<a href="https://colab.research.google.com/github/ksokoll/Destatis_Revenue_Analysis/blob/main/Langchain3_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Willkommen beim POC des Multiagenten-Frameworks basierend auf Langchain mit Majority-
Voting Funktion.

High-Level Funktionsweise des Agenten:

1. Aufgabe: Agent soll eines von 7 Szenarien auswählen. Hierzu stehen ihm verschiedene Tools zur Verfügung
2. Versuchsaufbau: Als Input erhält der Agent das Rechnungsbuchungs-Szenario mit 7 Beispielszenarien, sowie zusätzlichen Informationen
  zu den auf der Rechnung vorkommenden Bestellnummern, sowie einen Schriftverkehr mit einem fiktiven Besteller.
3. Genutzte Techniken: Regel-Prompt mit vordefiniertem Antwortformat, Langchain-Tools, Self-Refinement und Majority-Voting

Lessons Learned:

1. gpt-4o-mini als Modell reicht zum testen aus. Selbst mehrere hundert Durchläufe verursachen Kosten von maximal 30 Cent.
  Wechselt man auf gpt-4o erhöhen sich die Kosten massiv, sollte nur für wenige Tests genutzt werden oder wenn der Arbeitgeber die Token zahlt.

2. Das Majority-Voting hat die Ergebnisse verbessert, der Sweet Spot wie viele Votes durchgeführt werden sollen ist aber nicht gefunden. Ziel ist aus Performance-Gründen
  stets, nur so viele Durchläufe wie Nötig für eine gewisse erwartete Genauigkeit durchzuführen.

3. Eine Ausführung dauert teils 10-20 Minuten, je nach Anzahl der Durchläufe für das Voting. Dies macht dieses Setup nur für die Abarbeitung von
  nicht-Zeitkritischen Batchjobs interessant, oder für asynchrone Kommunikation via Mail. Für eine synchrone Kommunikation z.B. via Chat muss entweder
  ein anderes Framework benutzt, oder die Performance massiv verbessert werden.


"""

In [ ]:
!pip install langchain
!pip install langchain-community langchain-core
!pip install python-dotenv
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 

In [ ]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
import pandas as pd
from langchain.memory import ConversationBufferMemory
import re
from collections import Counter
from langchain.agents import initialize_agent, AgentType

In [ ]:
# Erstellen und Schreiben der .env-Datei zum testen
with open(".env", "w") as f:
    f.write(
        "OPENAI_API_KEY='<insert your key here>'\n"
    )

# Lädt die Umgebungsvariablen aus der .env-Datei
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Initialisiere das Sprachmodell (LLM)
llm = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", openai_api_key=openai_api_key)

llm2 = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", openai_api_key=openai_api_key)

<ipython-input-4-5c18a5bf2925>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", openai_api_key=openai_api_key)


In [ ]:
memory = ConversationBufferMemory()

<ipython-input-5-a33ea3a4cc0d>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [ ]:
# Main Agent System Prompt
agentSystemPrompt = """
    Du bist auf Regeln basierender Agent in einem Multiagenten-System. Deine Aufgabe ist die identifizierung eines vorliegenden Szenarios zu einer Rechnungsbuchung.

    - Schließe diesen Vorgang ausschließlich mit der Nennung des jeweiligen Szenarios ab.

    Antwortformat:
    - Wenn deine Antwort final ist, gib ausschließlich die Nummer des identifizierten Szenarios aus, z.B. "4".
    - Gib die finale Antwort ohne zusätzliche Erläuterungen und Erklärungen aus.

    Du musst immer folgende Schritte in dieser Reihenfolge ausführen.

  1. Prüfen der Korrespondenz
  2. Ausgabe der Szenarios
  3. Prüfen, ob die PO über genügend Budget verfügt, um die Rechnung zu Buchen
  4. Wahl des Szenarios und nutzung des final_check tools
  5. Evaluierung der Antwort des final_checks
  6. Finale Entscheidung für ein Szenario

    """

In [ ]:
# Scenarios

scenarioDatabase = """
1. Die PO verfügt über genügend Budget, um die Rechnung zu buchen.
2. Die PO verfügt nicht über genügend Budget, um die Rechnung zu buchen, und der Requester betont dass er nicht zuständig sei und es gibt keine weitere Information wer zuständig sein könnte.
3. Die PO verfügt nicht über genügend Budget, um die Rechnung zu buchen, und der Requester hat auf eine andere PO verwiesen, welche über genügend GR verfügt.
4. Die PO verfügt nicht über genügend Budget, um die Rechnung zu buchen, und der Requester hat auf eine andere PO verwiesen, welche auch nicht über genügend GR verfügt.
5. Die PO verfügt nicht über genügend Budget, es wurde eine neue PO genannt, welche jedoch ungültig ist.
6. Die PO verfügt nicht über genügend Budget, um die Rechnung zu buchen, aber der Requester hat auf eine andere Person verwiesen, die im Prozess weiterhelfen kann.
7. Szenario unklar bzw Szenario nicht definierbar

"""

In [ ]:
# Funktion für die PO-Datenbank

def check_PO_database(input_text: str):
    try:
        # Suche nach der PO im DataFrame
        result = df[df["PO"] == str(input_text).strip()]

        # Wenn kein Ergebnis gefunden wird, wirft es eine Exception
        if result.empty:
            raise ValueError(f"PO {input_text} wurde nicht gefunden.")

        return result

    except ValueError as e:
        # Gibt die Fehlermeldung zurück, falls die PO nicht gefunden wurde
        return str(e)

In [ ]:
# Example Data

PO = "4111234567"
Invoice_Amount = 450

# Example Agent Szenario
agentSzenario = f"""

Aktuelle Situation:

Rechnungssumme: {Invoice_Amount}€,

Bestellnummer: {PO}

Ergebnis der Bestellnummernprüfung für PO {PO}:

{check_PO_database("4111234567")}

"""

# Example PO & invoice Data
poDatabase = {
    "PO": ["4111234567", "4506543210", "4501973854"],
    "OriginalBudget": ["462", "242", "700"],
    "PostedInvoices": ["Invoice 15889 over 120€, Invoice 15890 over 122€", "Invoice 15112 over 50€", ""],
    "RemainingBudget": ["220", "120", "700"]
}

# Erstellen des DataFrames
df = pd.DataFrame(poDatabase)

# Example Workflow Correspondence
correspondence = """
17.09.2024: P2P-Team: 'Hi Gerald, leider reicht das Budget nicht aus, um die RE zu buchen.'
18.09.2024: Gerald: 'Kann auf eine andere PO: 4506543210 oder auf 4501973854, je nachdem wo noch was frei ist'
"""

# Bestellnummernsuche
bestellnummern = re.findall(r'\b\d{10}\b', correspondence)

# Bestellnummernprüfung und Ergebnis anhängen
correspondence += "\nErgebnis der automatischen Bestellnummernprüfungen:\n"

for bestellnummer in bestellnummern:
    po_data = df[df["PO"] == bestellnummer]
    if not po_data.empty:
        correspondence += f"\nBestellnummer {bestellnummer} gefunden:\n"
        correspondence += po_data.to_string(index=False)
    else:
        correspondence += f"\nBestellnummer {bestellnummer} nicht gefunden.\n"

# Ergebnis anzeigen
print(correspondence)



17.09.2024: P2P-Team: 'Hi Gerald, leider reicht das Budget nicht aus, um die RE zu buchen.'
18.09.2024: Gerald: 'Kann auf eine andere PO: 4506543210 oder auf 4501973854, je nachdem wo noch was frei ist'

Ergebnis der automatischen Bestellnummernprüfungen:

Bestellnummer 4506543210 gefunden:
        PO OriginalBudget         PostedInvoices RemainingBudget
4506543210            242 Invoice 15112 over 50€             120
Bestellnummer 4501973854 gefunden:
        PO OriginalBudget PostedInvoices RemainingBudget
4501973854            700                            700


In [ ]:
# Korrespondenz

def get_correspondence(item: str):
  return (correspondence)

correspondence_tool = Tool(
    name="Get Correspondence",
    func=get_correspondence,
    description="Gibt eine Zusammenfassung des bisherigen Austauschs mit dem Requester aus, auf die einzugehen ist."
)

In [ ]:
# Scenario-Tool

def get_scenarios(input: str):
  return scenarioDatabase

scenarios_tool = Tool(
    name="Get Scenarios",
    func=get_scenarios,
    description="Gibt dir die zur Verfügung stehenden Szenarien aus."
)

In [ ]:
# Finales Prüfungs-Tool

def final_check(input: str):
    global process_checked
    if process_checked:
      return("Der Prozess wurde bereits einmal geprüft, bitte schließe den Vorgang ab.")
    else:
      process_checked = True
      final_prompt = f"""
      Du bist ein Agent in einem Multiagenten-System und deine Aufgabe ist es, das ermittelte Szenario zu prüfen.

      Antwortformat: Umfassende Prüfung des Prozesses: Hat der Agent wirklich alles validiert, um das jeweilige Szenario garantieren zu können? Falls nicht, sende ihm eine klare Handlungsaufforderung.

      Wichtige Information: Es kann ausschließlich eines der unten stehenden Szenarien als Ergebnis herauskommen, nichts anderes.

      Budget der Bestellnummer:
      {check_PO_database("4111234567")}

      Parameter:
      {agentSzenario}

      Korrespondenz:
      {correspondence}

      Szenarien:
      {scenarioDatabase}



      Antwort des Hauptagenten: {input}
      """
      # Invoke LLM direkt ohne Tools
      rules_used = True
      return llm2.invoke([HumanMessage(content=final_prompt)]).content



final_tool = Tool(
    name="Final Check",
    func=final_check,
    description="Wenn du dir sicher bist was zu tun ist, wird hier eine finale Prüfung durchgeführt."
)

In [ ]:


# Liste der Tools, die der Agent nutzen kann
tools = [correspondence_tool, scenarios_tool, final_tool]

# Initialisiere den Agenten
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    memory=memory
)


In [ ]:
# Hier nur der Test eines einzigen Durchgangs mit verbose=True, um die Gedankengänge zu prüfen

global status
status = "Started"
global process_checked
process_checked = False

# Initialisiere den Agenten
agent2 = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

response = agent2.invoke(
    agentSystemPrompt + agentSzenario,
    max_iterations=50,
    timeout=120
    )

print(response)



> Entering new AgentExecutor chain...
Ich werde zunächst die Korrespondenz prüfen, um alle relevanten Informationen zu sammeln.  
Action: Get Correspondence  
Action Input: ""  
Observation: 
17.09.2024: P2P-Team: 'Hi Gerald, leider reicht das Budget nicht aus, um die RE zu buchen.'
18.09.2024: Gerald: 'Kann auf eine andere PO: 4506543210 oder auf 4501973854, je nachdem wo noch was frei ist'

Ergebnis der automatischen Bestellnummernprüfungen:

Bestellnummer 4506543210 gefunden:
        PO OriginalBudget         PostedInvoices RemainingBudget
4506543210            242 Invoice 15112 over 50€             120
Bestellnummer 4501973854 gefunden:
        PO OriginalBudget PostedInvoices RemainingBudget
4501973854            700                            700
Thought:Ich habe nun die Korrespondenz geprüft und relevante Informationen über die Budgetverfügbarkeit erhalten. Es gibt zwei alternative Bestellnummern, die verwendet werden können: 4506543210 und 4501973854. Ich werde nun die verfüg

In [ ]:
def get_vote():
  memory.chat_memory.clear()
  determined_scenarios = []

  for x in range(1): #Altes Majority-Voting System für die Gruppenvotes, steht gerade redundanterweise auf "1",
  #wurde durch das neue System mit Konvergenz unten ersetzt aber noch nicht angepasst

    global status
    status = "Started"
    global process_checked
    process_checked = False

    response = agent.invoke(
        agentSystemPrompt + agentSzenario,
        max_iterations=50,
        timeout=120
        )

  extracted_numbers = []
  ai_messages = [message for message in memory.chat_memory.messages if isinstance(message, AIMessage)]

  for ai_message in ai_messages:
    extracted_numbers.append(ai_message.content)

  numbers_as_integers = list(map(int, extracted_numbers))
  number_counts = Counter(numbers_as_integers)
  majority_vote = number_counts.most_common(1)[0][0]

  return(majority_vote)

In [ ]:
def monitor_convergence(num_groups, outer_iterations, convergence_threshold, min_iterations):
    converged = False
    iteration_results = []

    for outer_iteration in range(outer_iterations):
        group_votes = []

        # Führe get_vote() für jede Gruppe aus
        for _ in range(num_groups):  # z.B. 5 Gruppen
            group_vote = get_vote()
            group_votes.append(group_vote)

        # Berechne das finale Majority Vote über alle Gruppen hinweg
        final_majority_vote = Counter(group_votes).most_common(1)[0][0]
        iteration_results.append(final_majority_vote)

        # Konvergenzprüfung erst nach min_iterations Durchläufen
        if len(iteration_results) >= min_iterations:
            convergence_rate = sum([1 for i in iteration_results if i == final_majority_vote]) / len(iteration_results)
            print(f"Durchlauf {outer_iteration + 1}: Mehrheit = {final_majority_vote}, Konvergenzrate = {convergence_rate:.2f}")

            if convergence_rate >= convergence_threshold:
                print(f"Konvergenz erreicht bei Durchlauf {outer_iteration + 1} mit einer Konvergenzrate von {convergence_rate:.2f}")
                converged = True
                break
        else:
            # Ausgabe ohne Konvergenzrate
            print(f"Durchlauf {outer_iteration + 1}: Mehrheit = {final_majority_vote}")

    if not converged:
        print(f"Maximale Durchläufe ({outer_iterations}) erreicht ohne vollständige Konvergenz.")

# Testen des Tools mit 5 Durchläufen. Hier können wichtige Parameter wie die Konvergenzrate, sowie die inneren und äußeren Iterationen eingestellt werden
monitor_convergence(num_groups=1, outer_iterations=10, convergence_threshold=0.50, min_iterations=5)

Durchlauf 1: Mehrheit = 3
Durchlauf 2: Mehrheit = 3
Durchlauf 3: Mehrheit = 3
Durchlauf 4: Mehrheit = 3
Durchlauf 5: Mehrheit = 3, Konvergenzrate = 1.00
Konvergenz erreicht bei Durchlauf 5 mit einer Konvergenzrate von 1.00
